In [42]:
import pandas as pd
import time
import ray
from oanda_forex_utilities import *
import os
import warnings



In [43]:
account_details = get_oanda_account_credentials("demo_account_1.15")
account_id = account_details["account_id"]
access_token = account_details["access_token"]

print(account_id)
print(access_token)

101-003-18483320-018
87a216bb7b0e640e4af97a43499b58b9-4803472d0b66dd35b5d7541f3105caf5


In [44]:

ray.init()
@ray.remote
def automated_trade(currency_pair):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')  
        get_extraction_time("M5")
        time.sleep(1.5)
        state = True
        complete_list = []

        while state == True:
            df = get_candle_data(account_id, access_token, currency_pair, "M5", 500)
            df_longer_time = get_candle_data(account_id, access_token, currency_pair, "M30", 500)

            df_with_indicators = get_macd_indicator(df)
            df_with_indicators = get_adx_indicator(df_with_indicators)
            df_with_indicators = get_ema_indicator(df_with_indicators, window = 8)
            df_with_indicators = get_ema_indicator(df_with_indicators, window = 13)
            df_with_indicators = get_ema_indicator(df_with_indicators, window = 21)
            df_with_indicators_complete = df_with_indicators[df_with_indicators["complete"] == True]

            df_longer_time_with_indicators = get_ema_indicator(df_longer_time, window = 8)
            df_longer_time_with_indicators = get_ema_indicator(df_longer_time_with_indicators, window = 13)
            df_longer_time_with_indicators = get_ema_indicator(df_longer_time_with_indicators, window = 21)
            df_longer_time_with_indicators = get_ema_indicator(df_longer_time_with_indicators, window = 50)
            df_longer_time_with_indicators = get_rsi_indicator(df_longer_time_with_indicators)
            df_longer_time_with_indicators = get_adx_indicator(df_longer_time_with_indicators)
            
            if df_with_indicators_complete["time"].iloc[-1] not in complete_list:
                complete_list.append(df_with_indicators_complete["time"].iloc[-1])

                if (
                        (df_with_indicators_complete["macd_signal_diff"].iloc[-1] < 0) and
                        (df_with_indicators_complete["macd_signal_diff"].iloc[-2] < 0) and      
                        (df_with_indicators_complete["macd_signal_diff"].iloc[-3] >= 0) and

                        (df_with_indicators_complete["price_close"].iloc[-1] < df_with_indicators_complete["ema_8"].iloc[-1]) and
                        (df_with_indicators_complete["price_close"].iloc[-1] < df_with_indicators_complete["ema_13"].iloc[-1]) and
                        (df_with_indicators_complete["price_close"].iloc[-1] < df_with_indicators_complete["ema_21"].iloc[-1]) and

                        (df_with_indicators_complete["price_close"].iloc[-2] < df_with_indicators_complete["ema_8"].iloc[-2]) and
                        (df_with_indicators_complete["price_close"].iloc[-2] < df_with_indicators_complete["ema_13"].iloc[-2]) and
                        (df_with_indicators_complete["price_close"].iloc[-2] < df_with_indicators_complete["ema_21"].iloc[-2]) and

                        (df_longer_time_with_indicators["price_close"].iloc[-1] < df_longer_time_with_indicators["ema_8"].iloc[-1]) and
                        (df_longer_time_with_indicators["price_close"].iloc[-1] < df_longer_time_with_indicators["ema_13"].iloc[-1]) and
                        (df_longer_time_with_indicators["price_close"].iloc[-1] < df_longer_time_with_indicators["ema_21"].iloc[-1]) and
                        (df_longer_time_with_indicators["price_close"].iloc[-1] < df_longer_time_with_indicators["ema_50"].iloc[-1]) and

                        (df_longer_time_with_indicators["price_close"].iloc[-2] < df_longer_time_with_indicators["ema_8"].iloc[-2]) and
                        (df_longer_time_with_indicators["price_close"].iloc[-2] < df_longer_time_with_indicators["ema_13"].iloc[-2]) and
                        (df_longer_time_with_indicators["price_close"].iloc[-2] < df_longer_time_with_indicators["ema_21"].iloc[-2]) and
                        (df_longer_time_with_indicators["price_close"].iloc[-2] < df_longer_time_with_indicators["ema_50"].iloc[-2]) and
                    
                        (df_with_indicators_complete["adx"].iloc[-1] >= 20) and
                        (df_longer_time_with_indicators["adx"].iloc[-1] >= 20) and
#                         (df_longer_time_with_indicators["rsi"].iloc[-1] <= 30) and
                        (df_with_indicators_complete["time"].iloc[-1].time() >= datetime.time(6, 00)) and
                        (df_with_indicators_complete["time"].iloc[-1].time() <= datetime.time(18, 00)) and
                        
                        (get_position_details(account_id, access_token, currency_pair)["short"] == 0)


                    ):
                    print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " SELL ORDER FILLED")
                    if currency_pair in ["EUR_JPY", "AUD_JPY", "NZD_JPY", "USD_JPY", "GBP_JPY"]:
                        stop_loss = 1000
                    else:
                        stop_loss = 10
                    create_market_order_with_sl_tp(account_id, access_token, currency_pair, "SELL", 5000,
                                              round(float(df_with_indicators_complete["price_close"].iloc[-1]) + stop_loss/10000,5),
                                              round(float(df_with_indicators_complete["price_close"].iloc[-1]) - stop_loss/10000,5))


                elif(
                        (df_with_indicators_complete["macd_signal_diff"].iloc[-1] > 0) and
                        (df_with_indicators_complete["macd_signal_diff"].iloc[-2] > 0) and
                        (df_with_indicators_complete["macd_signal_diff"].iloc[-3] <= 0) and

                        (df_with_indicators_complete["price_close"].iloc[-1] > df_with_indicators_complete["ema_8"].iloc[-1]) and
                        (df_with_indicators_complete["price_close"].iloc[-1] > df_with_indicators_complete["ema_13"].iloc[-1]) and
                        (df_with_indicators_complete["price_close"].iloc[-1] > df_with_indicators_complete["ema_21"].iloc[-1]) and

                        (df_with_indicators_complete["price_close"].iloc[-2] > df_with_indicators_complete["ema_8"].iloc[-2]) and
                        (df_with_indicators_complete["price_close"].iloc[-2] > df_with_indicators_complete["ema_13"].iloc[-2]) and
                        (df_with_indicators_complete["price_close"].iloc[-2] > df_with_indicators_complete["ema_21"].iloc[-2]) and

                        (df_longer_time_with_indicators["price_close"].iloc[-1] > df_longer_time_with_indicators["ema_8"].iloc[-1]) and
                        (df_longer_time_with_indicators["price_close"].iloc[-1] > df_longer_time_with_indicators["ema_13"].iloc[-1]) and
                        (df_longer_time_with_indicators["price_close"].iloc[-1] > df_longer_time_with_indicators["ema_21"].iloc[-1]) and
                        (df_longer_time_with_indicators["price_close"].iloc[-1] > df_longer_time_with_indicators["ema_50"].iloc[-1]) and
                        
                        (df_longer_time_with_indicators["price_close"].iloc[-2] > df_longer_time_with_indicators["ema_8"].iloc[-2]) and
                        (df_longer_time_with_indicators["price_close"].iloc[-2] > df_longer_time_with_indicators["ema_13"].iloc[-2]) and
                        (df_longer_time_with_indicators["price_close"].iloc[-2] > df_longer_time_with_indicators["ema_21"].iloc[-2]) and
                        (df_longer_time_with_indicators["price_close"].iloc[-2] > df_longer_time_with_indicators["ema_50"].iloc[-2]) and
    
                        (df_with_indicators_complete["adx"].iloc[-1] >= 20) and
                        (df_longer_time_with_indicators["adx"].iloc[-1] >= 20) and
#                         (df_longer_time_with_indicators["rsi"].iloc[-1] <= 70) and
                        (df_with_indicators_complete["time"].iloc[-1].time() >= datetime.time(6, 00)) and
                        (df_with_indicators_complete["time"].iloc[-1].time() <= datetime.time(18, 00)) and
                                            
                        (get_position_details(account_id, access_token, currency_pair)["long"] == 0)
                    ):
                    
                    print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " BUY ORDER FILLED")
                    if currency_pair in ["EUR_JPY", "AUD_JPY", "NZD_JPY", "USD_JPY", "GBP_JPY"]:
                        stop_loss = 1000
                    else:
                        stop_loss = 10
                    create_market_order_with_sl_tp(account_id, access_token, currency_pair, "BUY", 5000,
                                              round(float(df_with_indicators_complete["price_close"].iloc[-1]) - stop_loss/10000,5),
                                              round(float(df_with_indicators_complete["price_close"].iloc[-1]) + stop_loss/10000,5))
                    
                else:
                    print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " NOTHING HAPPENS")
            time.sleep(10)

2021-08-15 15:06:52,100	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8265


In [ ]:
for i in range(1, 20):
    try:
        ray.get([automated_trade.remote("EUR_USD"), automated_trade.remote("USD_JPY"),
                 automated_trade.remote("USD_CHF"), automated_trade.remote("GBP_USD"),
                 automated_trade.remote("NZD_CAD"), automated_trade.remote("GBP_JPY"),
                 automated_trade.remote("NZD_USD"), automated_trade.remote("EUR_GBP")])
    except Exception as e:
        print (e)
        print ('Restarting!')
        ray.shutdown()
        continue
    else:
        break

(pid=98305) 2021-08-13 20:55:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=98303) 2021-08-13 20:55:00+00:00 NZD_USD  NOTHING HAPPENS
(pid=98301) 2021-08-13 20:55:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=98306) 2021-08-13 20:55:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=98309) 2021-08-13 20:55:00+00:00 NZD_CAD  NOTHING HAPPENS
(pid=98307) 2021-08-13 20:55:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=98304) 2021-08-13 20:55:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=98308) 2021-08-13 20:55:00+00:00 USD_JPY  NOTHING HAPPENS


In [41]:
ray.shutdown()